# decorator, 도대체 어떻게 쓰는걸까?

## 1. Without decorator **@**

In [38]:
def i_am_decorator(real_func):
    print('decoration을 위한 outer 함수입니다')
    
    def wrapper():
        print("=====@@@@@=====")
        real_func()
        print("=====@@@@@=====")
        
    return wrapper

def i_need_decorator():
    print("I need a decorator. Do you Understand?")

In [39]:
i_am_decorator(i_need_decorator) ## 

decoration을 위한 outer 함수입니다


<function __main__.i_am_decorator.<locals>.wrapper()>

In [40]:
i_am_decorator(i_need_decorator)()

decoration을 위한 outer 함수입니다
=====@@@@@=====
I need a decorator. Do you Understand?
=====@@@@@=====


#### 0) 이렇게 보니 쉽지 않은가? 이제부터 직관적으로 설명을 시작해보겠다.

#### 1) `i_need_decorator`라는 함수를 인자로 받는 `i_am_decorator`라는 함수를 실행하는 것이므로,  우선 `i_am_decorator`를 실행하는 것에 집중하자.

#### 2)  따라서, `i_am_decorator`의 가장 `outer`에 있는 print('decoration을 위한 outer 함수입니다')가 가장 먼저 실행된다

#### 3) 그 후, return을 실행해야 하는데 그것이 바로 **wrapper**이다.

#### 4) **wrapper**라는 함수를 사용하려고 하는데 괄호가 없네.? 라는 것을 상단의 실행 결과가 보여주고 있는 것이다.

#### 5) 그래서 아래에서 괄호를 넣고 실행해주었더니 **wrapper**라는 함수가 작동한다.

#### 6) **wrapper**라는 함수의 정의를 보면 **real_func**라는 함수를 (여기선, `i_need_decorator`) 사이에 두고 print()문이 두개 있다.

#### 7) `i_need_decorator`는 왜 괄호를 안해준거야? 라고 생각하시는 분이 혹시 있다면, **i_am_decorator**를 정의한 코드를 보라. 괄호가 없는 함수 자체를 받아 함수를 붙여 실행하고 있으니 말이다.

## 2. With decorator **@**

- 사실 실제 코딩에서는 1번처럼 쓰지 않는다. @decorator라는 기호로 보통 표현하기 때문이다.
- 1번의 예시를 decorator @ 기호를 써서 나타내보았다.

In [41]:
def i_am_decorator(real_func):
    print('decoration을 위한 outer 함수입니다')
    
    def wrapper():
        print("=====@@@@@=====")
        real_func()
        print("=====@@@@@=====")
        
    return wrapper

@i_am_decorator
def i_need_decorator():
    print("I need a decorator. Do you Understand?")
    


decoration을 위한 outer 함수입니다


In [42]:
i_need_decorator() # decorator로서 위 아래로 잘 감싸서 decorate 해주고 있다.

=====@@@@@=====
I need a decorator. Do you Understand?
=====@@@@@=====


- 실제로 이를 활용하는 간편한 여러가지 예시가 있다 (개인적으로는 활용해봤던 memory_profiler 모듈의 profile 함수가 있겠다.)
- 다음으론, 1번보단 조금 더 복잡한 예시를 한번 보겠다.

In [43]:
def generate_power(exponent):
    def wrapper(f):
        def inner(*args):
            result = f(*args)
            return exponent**result
        return inner
    return wrapper

@generate_power(2) # exponent
def raise_two(n):
    return n**2

print(raise_two(3)) # 2^(3**2)

512


#### 1) 여기선 `generate_power`가 위에서 썼던 `i_am_decorator`와 같다. 
#### 2) 우선 `generate_power(2)`라는 함수가 가장 먼저 실행되고 싶어하는 것이다.
#### 3) `generate_power(2)`의 반환값은 `wrapper`(함수)라서 `wrapper`를 실행하려고 한다.   
3-1)   `wrapper`를 실행하려고 하니 반환값으로서 `inner`(함수)를 실행하라고 한다.   
3-2)   그렇다면 `inner`를 실행한다. `exponent**f(*args)`가 그 반환결과값이다


#### 4) 여기서 주의할 점은 1.의 예시의 `real_func`과는 `raise_two`가 쓰이는 위치가 다소  다르다는 점이다. 
#### 5) 가장 바깥함수인 `generate_power`의 인자는 함수를 인자로 받고 있지 않고 `**연산` 수행이 가능한 피연산자 타입을 받아야 한다고 되어있기 때문이다.
#### 6) 따라서 `raise_two`는 가장 바깥의 함수의 인자로 들어가고 싶어도 들어갈 수 없다. 는 것이 아니라, `wrapper`가 함수의 인자로 들어간다고 생각하면 되겠다.


>problem

- 그럼 위 설명들을 이해했는지 직접 생각하고 실습해보며 테스트를 해보도록 하자.
    - 위 2.의 @decorator를 활용한 코드를 @ 기호 없이 함수들의 구성으로만 표현하려면 어떻게 표현하면 될까?
    - 생각해볼 한번 시간을 가지고 아래 정답 코드를 보자.

In [48]:
def generate_power(exponent):
    def wrapper(f): # wrapper
        def inner(*args):
            result = f(*args)
            return exponent**result
        return inner
    return wrapper

def raise_two(n):
    return n**2

generate_power(2)(raise_two)(3) # 정답

512

- 위 코드에서 정의하는 함수는 그대로이고, 맨 아랫줄 모드가 decorator를 빼고 그처럼 구현할 수 있는 코드이다..

In [45]:
# 위에서 언급한대로, generate_power는 인자로 exponent 라는 연산이 가능한 숫자 값이 필요
# 따라서, raise_two가 들어가는 위치는 wrapper()함수의 인자로서.
# wrapper()를 실행하면 raise_two가 실행되는 구조이니 raise_two의 인자가 필요.
